In [89]:
import requests
from urllib.request import urlopen
from bs4 import BeautifulSoup
import re
from datetime import datetime

In [ ]:
import configparser

config = configparser.ConfigParser()
config.read('./config.ini')

db_host = config['database']['host']
db_user = config['database']['user']
db_password = config['database']['password']
db_database = config['database']['database']
db_charset = config['database']['charset']


openai_key = config['openai']['OPENAI_KEY']

In [91]:
import sys

import csv
import pymysql

connection = pymysql.connect(
    host=db_host,
    user=db_user,
    password=db_password,
    database=db_database,
    charset=db_charset,
    cursorclass=pymysql.cursors.DictCursor
)
# db에 연결
cursor = connection.cursor()

current_time = datetime.now().strftime('%Y-%m-%d %H:%M:%S')

delete_query = "DELETE FROM event_summary WHERE card_company_id = 1"

cursor.execute(delete_query)
connection.commit()

In [83]:
import openai
import json
openai.api_key=openai_key
prompt = "입력된 데이터를 [] 사이에 주어진 key를 가지는 JSON 형식의 list로 요약하여 제공해 줘 [benefit_field, content]\\nbenefit_field는 혜택의 분야, content는 혜택 할인율 정보를 핵심만 요약하여 나타냄."
def Summary(contents):
    response = openai.ChatCompletion.create(
        model = 'gpt-3.5-turbo-0125',
        messages=[
            {"role": "system", "content":  prompt},
            {"role": "user", "content": contents},
        ],
    
        temperature=0
    )

    return response["choices"][0]["message"]["content"]

In [92]:
html=urlopen('https://card.kbcard.com/BON/DVIEW/HBBMCXCRVNEC0002')
bs=BeautifulSoup(html,'html.parser')
links = bs.findAll('li', {'class':'col c6'})
for link in links:
    event_link='https://card.kbcard.com/BON/DVIEW/HBBMCXCRVNEC0002?mainCC=a&eventNum='+link.a.get('href').split('\'')[1]
    print(event_link)
    html=urlopen(event_link)
    bs=BeautifulSoup(html,'html.parser')

    field=bs.find('div', {'class':'title'}).b.text
    print(field)

    dates = bs.find('dl', {'class':'column2'}).dd.text.replace(" ","").split("-")
    startdate = datetime.strptime(dates[0], '%Y.%m.%d')
    enddate = datetime.strptime(dates[1], '%Y.%m.%d')

    contents = bs.find('ul', {'class':'listNum3'}).text.replace("\t","").replace("\n\n","\n")
    summary_contents=Summary(contents)

    data = json.loads(summary_contents)
    for item in data:
        benefit_field = field+" "+item['benefit_field']
        content = item['content']
        cursor.execute("""
            INSERT INTO `event_summary`(event_field,event_contents,expire_date,start_date,card_company_id)
            VALUES  ('%s', '%s', '%s', '%s', '%d') 
            """%(benefit_field,content,startdate,enddate,1))
        connection.commit()
    print()

https://card.kbcard.com/BON/DVIEW/HBBMCXCRVNEC0002?mainCC=a&eventNum=6458
AK플라자

https://card.kbcard.com/BON/DVIEW/HBBMCXCRVNEC0002?mainCC=a&eventNum=6445
농협하나로마트

https://card.kbcard.com/BON/DVIEW/HBBMCXCRVNEC0002?mainCC=a&eventNum=5488
롯데마트

https://card.kbcard.com/BON/DVIEW/HBBMCXCRVNEC0002?mainCC=a&eventNum=5487
홈플러스

https://card.kbcard.com/BON/DVIEW/HBBMCXCRVNEC0002?mainCC=a&eventNum=5492
이마트

https://card.kbcard.com/BON/DVIEW/HBBMCXCRVNEC0002?mainCC=a&eventNum=6452
티몬

https://card.kbcard.com/BON/DVIEW/HBBMCXCRVNEC0002?mainCC=a&eventNum=6456
네이버쇼핑

https://card.kbcard.com/BON/DVIEW/HBBMCXCRVNEC0002?mainCC=a&eventNum=6455
쿠팡

https://card.kbcard.com/BON/DVIEW/HBBMCXCRVNEC0002?mainCC=a&eventNum=6382
시크(CHIC)

https://card.kbcard.com/BON/DVIEW/HBBMCXCRVNEC0002?mainCC=a&eventNum=6459
AK플라자 광명점

https://card.kbcard.com/BON/DVIEW/HBBMCXCRVNEC0002?mainCC=a&eventNum=6442
하이마트



In [90]:
event_link='https://card.kbcard.com/BON/DVIEW/HBBMCXCRVNEC0002?mainCC=a&eventNum=6445'
print(event_link)
html=urlopen(event_link)
bs=BeautifulSoup(html,'html.parser')
field=bs.find('div', {'class':'title'}).b.text
# name=bs.find('div', {'class':'title'}).h2.text
title = bs.find('div', {'class':'title'}).b.text+" "+bs.find('div', {'class':'title'}).h2.text
print(title)
dates = bs.find('dl', {'class':'column2'}).dd.text.replace(" ","").split("-")
startdate = datetime.strptime(dates[0], '%Y.%m.%d')
enddate = datetime.strptime(dates[1], '%Y.%m.%d')

contents = bs.find('ul', {'class':'listNum3'}).text.replace("\t","").replace("\n\n","\n")
summary_contents=Summary(contents)

data = json.loads(summary_contents)
for item in data:
    benefit_field = field+" "+item['benefit_field']
    content = item['content']
    print(f"Benefit Field: {benefit_field}")
    print(f"Content: {content}\n")
    cursor.execute("""
        INSERT INTO `event_summary`(event_field,event_contents,expire_date,start_date,card_company_id)
        VALUES  ('%s', '%s', '%s', '%s', '%d') 
        """%(benefit_field,content,startdate,enddate,1))
    connection.commit()



https://card.kbcard.com/BON/DVIEW/HBBMCXCRVNEC0002?mainCC=a&eventNum=6445
농협하나로마트 인기품목 현장할인
Benefit Field: 농협하나로마트 할인 혜택
Content: 농축수산물, 필수 가공품, 생필품 등 다양한 품목에 최대 20% 할인

Benefit Field: 농협하나로마트 무이자할부 혜택
Content: 개인신용카드로 건당 5만원 이상 구매 시 2~3개월 무이자할부 혜택



In [93]:
cursor.close()
connection.close()